In [33]:
import pandas as pd
pd.options.display.float_format = '{:,.2f}'.format
pd.set_option('display.max_columns', 1000)
pd.set_option('display.max_rows', 100)

import qgrid

In [75]:
file = '/Users/brendan/Desktop/Python/BoostedAI/quarterly_data/AAPL.csv'

In [35]:
df = pd.read_csv(file)

In [36]:
df.head(2)

,Unnamed: 0,Calendar_Year,Quarter,Mkt Cap,Debt to Equity,Debt to Assets,Revenue per Share,NI per Share,outstandingShares,Revenue,Gross Profit,R&D Expenses,Op Expenses,Op Income,Net Income,Cash,Receivables,Inventory,Cur Assets,Tax Assets,LT Assets,Int Assets,Total Assets,Payables,Deferred Revenue Current,Deferred Revenue LongTerm,Cur Liab,LT Debt,LT Liab,Total Liab,SH Equity,cash_flow Operations,cash_flow Investing,cash_flow Financing,CAPEX,Fcash_flow,Dividends Paid,Gross Profit Margin,Op Margin,Int Coverage,Net Profit Margin,Dividend Yield,Current Ratio,Operating Cycle,Days of AP Outstanding,Cash Conversion Cycle,ROA,ROE,ROCE,PE,PS,PB,Price To FCF,PEG,EPS,Ticker
0,2022-06-25,2021,Q4,"14,937.81",1.08,0.74,38.49,1.31,62.84,"2,396.97","1,072.12",0.00,959.65,112.46,81.67,138.73,957.80,"4,778.85","6,057.78",0.00,"5,944.70",642.12,"12,002.48","3,942.39",0.00,0.00,"5,052.57","3,517.70","4,040.23","9,092.80","2,909.68",187.33,-98.43,-95.37,-98.66,88.67,0.00,0.45,0.05,11.59,0.03,NaN,1.21,345.89,266.43,79.46,0.01,0.03,0.02,45.73,6.23,4.78,168.47,-21.03,1.31,AAP
1,2022-03-26,2021,Q3,"13,460.18",1.00,0.73,41.70,2.70,63.35,"2,621.23","1,182.45",0.00,953.26,229.20,169.81,601.43,782.78,"4,659.02","6,275.48",0.00,"5,918.73",651.22,"12,194.21","3,922.01",0.00,0.00,"5,180.31","3,371.97","3,885.61","9,065.92","3,128.29",148.78,-61.16,-292.30,-61.41,87.37,-62.95,0.45,0.09,26.69,0.06,0.00,1.24,310.38,234.63,75.76,0.01,0.05,0.03,19.82,5.14,4.21,154.06,-8.01,2.70,AAP


## Quality

In [37]:
def asset_growth(ticker):
    """Asset growth is defined as the YoY % change in quarterly total assets.
    Higher is worse, so multiply by -1"""
    df = pd.read_csv(f'/Users/brendan/Desktop/Python/BoostedAI/quarterly_data/{ticker}.csv')
    df = df[::-1]
    df['Asset_growth'] = (df.groupby('Quarter')['Total Assets'].pct_change()[1:]*-1)*100
    return df[['Calendar_Year', 'Quarter','Total Assets','Asset_growth']]

In [38]:
asset_growth('AAPL').tail(1)

,Calendar_Year,Quarter,Total Assets,Asset_growth
0,2022,Q3,"336,309.00",-1.96


### Account clarity

`Account Clarity`: 
+ 12 Quarter Avg of:
+ current OP assets - OP liabilities
+ less 4 quarter ago OP assets - Op liabilities
+ scaled by average assets over the same period

where:
+ Operating Assets = Total Assets - cash
+ Operating Liability = Total Assets - debt - minority interests - common & pref. stock 

In [39]:
df.head(1)

,Unnamed: 0,Calendar_Year,Quarter,Mkt Cap,Debt to Equity,Debt to Assets,Revenue per Share,NI per Share,outstandingShares,Revenue,Gross Profit,R&D Expenses,Op Expenses,Op Income,Net Income,Cash,Receivables,Inventory,Cur Assets,Tax Assets,LT Assets,Int Assets,Total Assets,Payables,Deferred Revenue Current,Deferred Revenue LongTerm,Cur Liab,LT Debt,LT Liab,Total Liab,SH Equity,cash_flow Operations,cash_flow Investing,cash_flow Financing,CAPEX,Fcash_flow,Dividends Paid,Gross Profit Margin,Op Margin,Int Coverage,Net Profit Margin,Dividend Yield,Current Ratio,Operating Cycle,Days of AP Outstanding,Cash Conversion Cycle,ROA,ROE,ROCE,PE,PS,PB,Price To FCF,PEG,EPS,Ticker
0,2022-06-25,2021,Q4,"14,937.81",1.08,0.74,38.49,1.31,62.84,"2,396.97","1,072.12",0.00,959.65,112.46,81.67,138.73,957.80,"4,778.85","6,057.78",0.00,"5,944.70",642.12,"12,002.48","3,942.39",0.00,0.00,"5,052.57","3,517.70","4,040.23","9,092.80","2,909.68",187.33,-98.43,-95.37,-98.66,88.67,0.00,0.45,0.05,11.59,0.03,NaN,1.21,345.89,266.43,79.46,0.01,0.03,0.02,45.73,6.23,4.78,168.47,-21.03,1.31,AAP


In [50]:
def operating_assets(ticker, window):
    df = pd.read_csv(f'/Users/brendan/Desktop/Python/BoostedAI/quarterly_data/{ticker}.csv')
    df = df[::-1]
    df['current assets'] = (df['Total Assets'].rolling(window).sum() - df['Cash'].rolling(window).sum())
    return df['current assets']

def operating_liability(ticker, window):
    df = pd.read_csv(f'/Users/brendan/Desktop/Python/BoostedAI/quarterly_data/{ticker}.csv')
    df = df[::-1]
    df['current liabilities'] = (df['Total Assets'].rolling(window).sum() - df['Cash'].rolling(window).sum())
    return df

def operating_assets_12q_avg(ticker):
    df = pd.read_csv(f'/Users/brendan/Desktop/Python/BoostedAI/quarterly_data/{ticker}.csv')
    df = df[::-1]
    df['operating_assets_12q_avg'] = operating_assets(ticker, 12)
    return df[['Calendar_Year', 'Quarter', 'Mkt Cap', 'Ticker','operating_assets_12q_avg']]

def account_clarity(ticker):
    """12 Quarter Avg of:
    current OP assets - OP liabilities
    less 4 quarter ago OP assets - Op liabilities
    scaled by average assets over the same period 
     -------------------------------------------
     
     Operating Assets = Total Assets - cash
     Operating Liability = Total Assets - debt - minority interests - common & pref. stock 
     """
    df = pd.read_csv(f'/Users/brendan/Desktop/Python/BoostedAI/quarterly_data/{ticker}.csv')
    df = df[::-1]
    avg_assets_12q = df['Total Assets'].rolling(12).mean()
    assets_4q  = df['Total Assets'].shift(4) - df['']


In [51]:
operating_assets_12q_avg('AAPL').tail()

,Calendar_Year,Quarter,Mkt Cap,Ticker,operating_assets_12q_avg
4,2021,Q3,"2,213,535.58",AAPL,"3,597,583.00"
3,2021,Q4,"2,422,287.79",AAPL,"3,573,833.00"
2,2022,Q1,"2,955,919.62",AAPL,"3,588,957.00"
1,2022,Q2,"2,863,962.04",AAPL,"3,607,511.00"
0,2022,Q3,"2,289,642.85",AAPL,"3,644,609.00"


### Cash Operating Profit Growth

`Cash Operating Profit`:
+ GAAP EBIT

In [76]:
def cash_op_profit_growth(ticker):
    df = pd.read_csv(file)
    df = df[::-1]
    df['Op Income YoY'] = (df.groupby('Quarter')['Op Income'].pct_change())*100
    return df[['Calendar_Year', 'Quarter', 'Mkt Cap', 'Op Income', 'Op Income YoY' ]][::-1]

In [124]:
cash_op_profit_growth('AAPL').head()

,Calendar_Year,Quarter,Mkt Cap,Op Income,Op Income YoY
0,2022,Q3,"2,289,642.85","23,076.00",-4.35
1,2022,Q2,"2,863,962.04","29,979.00",9.00
2,2022,Q1,"2,955,919.62","41,488.00",23.72
3,2021,Q4,"2,422,287.79","23,786.00",60.99
4,2021,Q3,"2,213,535.58","24,126.00",84.29


### Working Capital Accruals

`working capital accruals`:
+ Trailing 4 quarters:
    + change in receivables +
    + change in inventories +
    + change in accts payable +
    + change in tax accruals +
    + change in Misc assets and liabilities
+ scaled by avg Total Assets

In [125]:
df.head(2)

,Unnamed: 0,Calendar_Year,Quarter,Mkt Cap,Debt to Equity,Debt to Assets,Revenue per Share,NI per Share,outstandingShares,Revenue,Gross Profit,R&D Expenses,Op Expenses,Op Income,Net Income,Cash,Receivables,Inventory,Cur Assets,Tax Assets,LT Assets,Int Assets,Total Assets,Payables,Deferred Revenue Current,Deferred Revenue LongTerm,Cur Liab,LT Debt,LT Liab,Total Liab,SH Equity,cash_flow Operations,cash_flow Investing,cash_flow Financing,CAPEX,Fcash_flow,Dividends Paid,Gross Profit Margin,Op Margin,Int Coverage,Net Profit Margin,Dividend Yield,Current Ratio,Operating Cycle,Days of AP Outstanding,Cash Conversion Cycle,ROA,ROE,ROCE,PE,PS,PB,Price To FCF,PEG,EPS,Ticker
0,2022-06-25,2021,Q4,"14,937.81",1.08,0.74,38.49,1.31,62.84,"2,396.97","1,072.12",0.00,959.65,112.46,81.67,138.73,957.80,"4,778.85","6,057.78",0.00,"5,944.70",642.12,"12,002.48","3,942.39",0.00,0.00,"5,052.57","3,517.70","4,040.23","9,092.80","2,909.68",187.33,-98.43,-95.37,-98.66,88.67,0.00,0.45,0.05,11.59,0.03,NaN,1.21,345.89,266.43,79.46,0.01,0.03,0.02,45.73,6.23,4.78,168.47,-21.03,1.31,AAP
1,2022-03-26,2021,Q3,"13,460.18",1.00,0.73,41.70,2.70,63.35,"2,621.23","1,182.45",0.00,953.26,229.20,169.81,601.43,782.78,"4,659.02","6,275.48",0.00,"5,918.73",651.22,"12,194.21","3,922.01",0.00,0.00,"5,180.31","3,371.97","3,885.61","9,065.92","3,128.29",148.78,-61.16,-292.30,-61.41,87.37,-62.95,0.45,0.09,26.69,0.06,0.00,1.24,310.38,234.63,75.76,0.01,0.05,0.03,19.82,5.14,4.21,154.06,-8.01,2.70,AAP


In [112]:
def working_cap_acrual(ticker):
    df = pd.read_csv(file)
    df = df[::-1]
    df['chg_rec'] = df.groupby('Quarter')['Receivables'].diff().fillna(0)
    df['chg_inv'] = df.groupby('Quarter')['Inventory'].diff().fillna(0)
    df['chg_pay'] = df.groupby('Quarter')['Payables'].diff().fillna(0)
    df['chg_tax'] = df.groupby('Quarter')['Tax Assets'].diff().fillna(0)
    df['chg_misc'] = df.groupby('Quarter')['Int Assets'].diff().fillna(0)
    df['avg_assets'] = df['Total Assets'].rolling(4).mean()
    df['workCapAcc'] = ((df['chg_rec'] + df['chg_inv'] + df['chg_pay'] + df['chg_tax'] + df['chg_misc'])/df['avg_assets'])*100
    return df[['Calendar_Year', 'Quarter', 'Ticker','Mkt Cap', 
               'chg_rec', 'chg_inv', 'chg_pay', 'chg_tax', 'chg_misc',
               'workCapAcc','Receivables', 'Inventory', 'Payables', 'Tax Assets', 'Total Assets']][::-1]
    

In [114]:
working_cap_acrual('AVGO').head()

,Calendar_Year,Quarter,Ticker,Mkt Cap,chg_rec,chg_inv,chg_pay,chg_tax,chg_misc,workCapAcc,Receivables,Inventory,Payables,Tax Assets,Total Assets
0,2022,Q3,AAPL,"2,289,642.85","8,334.00",255.00,"7,934.00",0.00,0.00,4.66,"42,242.00","5,433.00","48,343.00",0.00,"336,309.00"
1,2022,Q2,AAPL,"2,863,962.04","12,364.00",241.00,"12,555.00",0.00,0.00,7.12,"45,400.00","5,460.00","52,682.00",0.00,"350,662.00"
2,2022,Q1,AAPL,"2,955,919.62","6,633.00",903.00,"10,516.00",0.00,0.00,5.16,"65,253.00","5,876.00","74,362.00",0.00,"381,191.00"
3,2021,Q4,AAPL,"2,422,287.79","14,061.00","2,519.00","12,467.00",0.00,0.00,8.47,"51,506.00","6,580.00","54,763.00",0.00,"351,002.00"
4,2021,Q3,AAPL,"2,213,535.58","1,833.00","1,200.00","5,084.00",0.00,0.00,2.41,"33,908.00","5,178.00","40,409.00",0.00,"329,840.00"


### Net External Financing

`Net External Financing`:
+ 4 Qtr trailing Cash Flow from Financing Activities less
+ 4 Qtr Trailing Other Financing Activities
+ divided by the avg 4Q trailing Total Assetes

+ Higher reliance on external financing is wrose, so multiple by -1

+ If the above Numerator items are NA:
    + Sales of Common/Preferred Stock
    + (less) Purchase of Common?Preferred stock
    + (less) cash Dividends
    + (plus) Debt Issuance

In [116]:
df.head(3)

,Unnamed: 0,Calendar_Year,Quarter,Mkt Cap,Debt to Equity,Debt to Assets,Revenue per Share,NI per Share,outstandingShares,Revenue,Gross Profit,R&D Expenses,Op Expenses,Op Income,Net Income,Cash,Receivables,Inventory,Cur Assets,Tax Assets,LT Assets,Int Assets,Total Assets,Payables,Deferred Revenue Current,Deferred Revenue LongTerm,Cur Liab,LT Debt,LT Liab,Total Liab,SH Equity,cash_flow Operations,cash_flow Investing,cash_flow Financing,CAPEX,Fcash_flow,Dividends Paid,Gross Profit Margin,Op Margin,Int Coverage,Net Profit Margin,Dividend Yield,Current Ratio,Operating Cycle,Days of AP Outstanding,Cash Conversion Cycle,ROA,ROE,ROCE,PE,PS,PB,Price To FCF,PEG,EPS,Ticker
0,2022-06-25,2021,Q4,"14,937.81",1.08,0.74,38.49,1.31,62.84,"2,396.97","1,072.12",0.00,959.65,112.46,81.67,138.73,957.80,"4,778.85","6,057.78",0.00,"5,944.70",642.12,"12,002.48","3,942.39",0.00,0.00,"5,052.57","3,517.70","4,040.23","9,092.80","2,909.68",187.33,-98.43,-95.37,-98.66,88.67,0.00,0.45,0.05,11.59,0.03,NaN,1.21,345.89,266.43,79.46,0.01,0.03,0.02,45.73,6.23,4.78,168.47,-21.03,1.31,AAP
1,2022-03-26,2021,Q3,"13,460.18",1.00,0.73,41.70,2.70,63.35,"2,621.23","1,182.45",0.00,953.26,229.20,169.81,601.43,782.78,"4,659.02","6,275.48",0.00,"5,918.73",651.22,"12,194.21","3,922.01",0.00,0.00,"5,180.31","3,371.97","3,885.61","9,065.92","3,128.29",148.78,-61.16,-292.30,-61.41,87.37,-62.95,0.45,0.09,26.69,0.06,0.00,1.24,310.38,234.63,75.76,0.01,0.05,0.03,19.82,5.14,4.21,154.06,-8.01,2.70,AAP
2,2021-12-25,2021,Q2,"13,327.76",0.95,0.72,40.92,2.76,65.21,"2,649.41","1,189.25",0.00,944.32,244.93,178.70,604.64,931.76,"4,450.45","6,161.26",0.00,"5,702.91",658.54,"11,864.17","3,750.84",0.00,0.00,"4,955.23","3,188.37","3,712.41","8,667.64","3,196.53",446.23,-57.42,-459.75,-58.69,387.54,-64.83,0.45,0.09,29.49,0.07,0.00,1.27,300.24,234.67,65.57,0.02,0.05,0.04,18.65,5.03,4.04,34.39,0.26,2.76,AAP


In [126]:
def net_external_finance(ticker):
    df = pd.read_csv(file)
    df = df[::-1]
    df['trailing_CFF'] = df.groupby('Quarter')['cash_flow Financing'].diff().fillna(0)
    #df['trailing_OCF'] = df.groupby('Quarter')['Receivables'].diff().fillna(0)
    df['avg_total_assets'] = df['Total Assets'].rolling(4).mean()
    df['netExternFinance'] = df['trailing_CFF'] /df['avg_total_assets'] *100*-1
    return df[['Calendar_Year', 'Quarter', 'Ticker','Mkt Cap',
               'netExternFinance',
               'cash_flow Financing', 'trailing_CFF','Total Assets', 'avg_total_assets'
              ]][::-1]

In [127]:
net_external_finance('AVGO').head()

,Calendar_Year,Quarter,Ticker,Mkt Cap,netExternFinance,cash_flow Financing,trailing_CFF,Total Assets,avg_total_assets
0,2022,Q3,AAPL,"2,289,642.85",-0.55,"-27,445.00","1,951.00","336,309.00","354,791.00"
1,2022,Q2,AAPL,"2,863,962.04",4.82,"-28,351.00","-17,025.00","350,662.00","353,173.75"
2,2022,Q1,AAPL,"2,955,919.62",-1.17,"-28,159.00","4,090.00","381,191.00","349,797.75"
3,2021,Q4,AAPL,"2,422,287.79",-0.28,"-20,382.00",975.00,"351,002.00","343,013.50"
4,2021,Q3,AAPL,"2,213,535.58",3.06,"-29,396.00","-10,280.00","329,840.00","336,235.00"
